# Top

Before `git add`:

* Edit, Clear Outputs of All Cells

In [ ]:
import fullcontrol as fc

# Utility Functions

## GCode Banner

In [ ]:
def gcode_banner(text):
    init = []
    init.append(fc.GcodeComment(text='#' * 80))
    init.append(fc.GcodeComment(text=f'# {text}'))
    init.append(fc.GcodeComment(text='#' * 80))
    return init

## Square Spiral

In [ ]:
def _spiral(outer_size, num_turns, cw):
    """
    Generate a square spiral centered on (0, 0) that reaches `outer_size` after `num_turns`.
    Returns a list of (x, y) coordinates.
    """
    if num_turns < 1:
        return [(0, 0)]

    num_segments = 4 * num_turns + 1

    step = outer_size / num_turns / 2
    x, y = 0, 0
    dx, dy = 1, 0  # initial direction: right
    coords = [(x, y)]

    for i in range(num_segments):
        f = (i // 2 + 1 * (i < num_segments - 1)) * step
        x += dx * f
        y += dy * f
        coords.append((x, y))
        if cw:
            dx, dy = dy, -dx  # rotate 90 degrees CW
        else:
            dx, dy = -dy, dx  # rotate 90 degrees CCW

    return coords

In [ ]:
# Inspiration from:
# spiralXY(centre: Point, start_radius: float, end_radius: float, start_angle: float, n_turns: float, segments: int, cw: bool=False) -> list
def squareSpiral(center, size: float, n_turns: int, cw=False):
    steps = []
    for p in _spiral(size, n_turns, cw):
        steps.append(fc.Point(x=p[0] + center.x, y=p[1] + center.y, z=center.z))
    return steps

In [ ]:
# Mini test

print(
    fc.transform(
        squareSpiral(fc.Point(x=10, y=10, z=.2), 4, 2, True),
        'gcode',
        controls=fc.GcodeControls(
            printer_name='generic',  # or ender_3
            initialization_data={
                'extrusion_width': .4,
                'extrusion_height': .2
            }
        )
    )
)

## Hop To

In [ ]:
def hop_to(x: float, y: float):
    """Similar to fc.travel_to(), but with retract."""
    steps = []

    steps.append(fc.PrinterCommand(id='retract'))  # Already performing a Z hop
    steps.append(fc.Extruder(on=False))
    #steps.append(fc.ManualGcode(text="G91 ; Set all axes to relative"))
    #steps.append(fc.PlotAnnotation(label='Up'))
    #steps.append(fc.ManualGcode(text=f"G0 Z{hop_distance} ; Move up"))

    #steps.append(fc.ManualGcode(text="G90 ; Set all axes to absolute"))
        
    steps.append(fc.Point(x=x, y=y))
    
    #steps.append(fc.ManualGcode(text="G91 ; Set all axes to relative"))
    #steps.append(fc.PlotAnnotation(label='Down'))
    #steps.append(fc.ManualGcode(text=f"G0 Z-{hop_distance} ; Move down"))
    #steps.append(fc.ManualGcode(text="G90 ; Set all axes to absolute"))
    #steps.append(fc.ManualGcode(text="M83 ; Extruder relative"))

    steps.append(fc.PrinterCommand(id='unretract'))
    steps.append(fc.Extruder(on=True))

    return steps

In [ ]:
# Mini test

print(
    fc.transform(
        [fc.Point(x=1, y=1, z=.2)] + hop_to(10, 20),
        'gcode',
        controls=fc.GcodeControls(
            printer_name='generic',  # or ender_3
            initialization_data={
                'extrusion_width': .4,
                'extrusion_height': .2
            }
        )
    )
)

# Features

## Hotend Temperature

```
steps.append(fc.Hotend(temp=220, wait=True))
```

## Bed Temperature

```
steps.append(fc.Buildplate(temp=60, wait=True))
```

## Extruder on/off

```
# Travel
steps.extends(fc.travel_to(fc.Point(...)))

# Manuel
steps.append(fc.Extruder(on=False))
```

## Extrusion size

```
steps.append(fc.ExtrusionGeometry(area_model='rectangle', width=0.4, height=0.2))

steps.append(fc.ExtrusionGeometry(area_model='circle', diameter=1))
```

## Print Speed

```
steps.append(fc.Printer(print_speed=750))
```

## Travel Speed

```
steps.append(fc.Printer(travel_speed=2000))
```

## Fan

```
steps.append(fc.Fan(speed_percent=50))
```

## Retraction

```
steps.append(fc.PrinterCommand(id='retract'))

steps.append(fc.PrinterCommand(id='unretract'))
```

## GCode

### Comments

```
steps.append(fc.GcodeComment(text='This is a comment'))
```

### Custom commands

```
steps.append(fc.Printer(new_command={'beep': 'M300 S440 P1000 ; beep'}))
```

### Manual GCode

```
steps.append(fc.ManualGcode(text='G4 P2000 ; pause for 2 seconds'))
```

# Design

## Parameters

In [ ]:
# Parameters

#design_name = 'my_design'
nozzle_temp = 220
bed_temp = 60
print_speed = 900
travel_speed = 8000
#fan_percent = 100
#printer_name='ender_3' # generic / ultimaker2plus / prusa_i3 / ender_3 / cr_10 / bambulab_x1 / toolchanger_T0

EW = 0.6 # extrusion width
EH = 0.2 # extrusion height (and layer height)

## Simple

In [ ]:
steps = []

steps.append(fc.Extruder(on=False))
steps.append(fc.Point(x=10, y=10, z=.2))
steps.append(fc.Extruder(on=True))
steps.append(fc.Point(x=110))
steps.append(fc.Point(y=110))
steps.append(fc.Point(x=10, y=10))

steps

## Complex

### All in one

In [ ]:
steps = []

z = EH

steps.append(fc.Point(x=2, y=2, z=z))
steps.append(fc.Extruder(on=True))
steps.append(fc.Point(y=200))
steps.append(fc.Extruder(on=False))

pt = fc.Point(x=20, y=20)
steps.append(pt)

steps.append(fc.Extruder(on=True))
steps.extend(fc.rectangleXY(pt, 100, 10, cw=True))
steps.append(fc.Extruder(on=False))

pt = fc.Point(x=40, y=100)
steps.append(pt)
steps.append(fc.Extruder(on=True))
steps.extend(fc.spiralXY(pt, 1, 10, 0, 10, 1000, True))
steps.append(fc.Extruder(on=False))

pt = fc.Point(x=20, y=40)
steps.append(pt)
steps.append(fc.Extruder(on=True))
steps.extend(fc.squarewaveXY(pt, fc.Vector(x=1, y=0, z=0), 10, 2, 30))
steps.append(fc.Extruder(on=False))

pt = fc.Point(x=100, y=100)
steps.append(pt)
steps.append(fc.Extruder(on=True))
steps.extend(squareSpiral(pt, 80, 20, True))
# steps

### Spiral

In [ ]:
steps = []

#steps.append(fc.Point(x=3, y=2, z=EH))
pt = fc.Point(x=100, y=100, z=EH)
steps.extend(hop_to(pt.x, pt.y))
steps.extend(fc.spiralXY(pt, .1, 50, 0, 10, 2000, True))

## Calibrations

### Bed Square Spiral

In [ ]:
steps = []

p = fc.Point(x=110, y=110, z=EH)

steps.append(fc.PrinterCommand(id='retract'))
steps.append(p)
steps.append(fc.PrinterCommand(id='unretract'))
steps.append(fc.Extruder(on=True))
steps.extend(squareSpiral(p, 200, 20, cw=True))

### Width Tests

In [ ]:
steps = []

steps.append(fc.Extruder(on=False))
steps.append(fc.Point(x=5, y=10, z=EH))
steps.append(fc.Fan(speed_percent=0))  # Fan OFF for first layer

for i in range(15):
    w = (i+1)/10.0
    steps.append(fc.GcodeComment(text=f'Width = {w:.2f}'))
    steps.append(fc.Point(x=10+10*i, y=10))
    steps.append(fc.Extruder(on=True))
    steps.append(fc.ExtrusionGeometry(width=w))
    steps.append(fc.Point(y=110))
    steps.append(fc.Extruder(on=False))

Conclusion:

* On bed
    * EH = .2
    * EW [.6, 1.0]

### Retraction Tests

Default:

```
>>> M207
<< M207 S5.00 W13.00 F2400.00 Z0.20

>>> M208
<< M208 S0.00 W0.00 F2400.00
```

From Orca gcode:
* Z hop seems .4
* 3600 retract
* 2400 recover

In [ ]:
steps = []

steps.append(fc.Extruder(on=False))
steps.append(fc.Point(x=5, y=5, z=EH))

# Retraction: M207 F<feedrate> S<length>
# Unretraction: M208 F<feedrate> S<length>.  Length is addition to M207 S

# F: feedrate (units/min)  (3600 in gcode?)
# S: length (mm)  0 to 10 ?

steps.append(fc.ManualGcode(text=f"M207 Z0.4 ; retraction z hop"))
steps.append(fc.ManualGcode(text=f"M207 F3600 ; retraction speed"))
steps.append(fc.ManualGcode(text=f"M208 F2400 ; un-retraction speed"))

for j in range(1):  # Feedrate
    #steps.append(fc.ManualGcode(text=f"M207 F{3600 + 10 * j:d} ; retraction speed"))
    #steps.append(fc.ManualGcode(text=f"M208 F{3600 + 10 * j:d} ; un-retraction speed"))
    for i in range(80):  # Length
        steps.append(fc.ManualGcode(text=f"M207 S{.1*(i+1):.2f} ; retraction length"))
        #steps.append(fc.ManualGcode(text=f"M208 S{.1 * i:.2f} ; un-retraction length"))
        steps.extend(hop_to(10 + 2 * i, 10 + 15 * j))
        steps.append(fc.Point(y=20 + 15 * j))

### Linear Advance K Factor

M900 Kx ; x (from 0 to 1)

In [ ]:
steps = []  # TBD continue...

# Visualization

In [ ]:
fc.transform(
    steps,
    'plot',
    fc.PlotControls(
        #neat_for_publishing=True,
        style='line',
        #style='tube',
        zoom=.9
    )
)

# GCODE

## Basic

In [ ]:
gcode = fc.transform(
    steps,
    'gcode',
    controls=fc.GcodeControls(
        printer_name='generic',  # or ender_3
        initialization_data={
            'extrusion_width': EW,
            'extrusion_height': EH
        }
    )
)

print(gcode)

## Ender 3

In [ ]:
gcode = fc.transform(
    steps,
    'gcode',
    fc.GcodeControls(
        printer_name='ender_3',
        initialization_data={
            'primer': 'travel',  # front_lines_then_y
            'print_speed': print_speed,
            'nozzle_temp': nozzle_temp,
            'bed_temp': bed_temp,
            'fan_percent': 0,
            'extrusion_width': EW,
            'extrusion_height': EH},
        save_as='dummy',
        include_date=False
    )
)

for l in gcode.splitlines()[:50]:
    print(l)

## GCode Generate

In [ ]:
# Custom for Ender 3 + BLTouch
def to_gcode(steps, filename='fullcontrol'):
    init = []
    init.extend(gcode_banner('STARTING PROCEDURE'))
    init.append(fc.Printer(new_command={'beep': 'M300 S440 P1000 ; beep'}))
    init.append(fc.PrinterCommand(id='beep'))
    init.append(fc.ManualGcode(text='G90 ; use absolute coordinates'))
    init.append(fc.ManualGcode(text="M207 Z0.4 ; retraction z hop"))
    init.append(fc.ManualGcode(text="M207 F3600 ; retraction speed"))
    init.append(fc.ManualGcode(text="M207 S5 ; retraction length"))
    init.append(fc.ManualGcode(text="M208 F2400 ; un-retraction speed"))

    init.append(fc.Extruder(relative_gcode=True))
    init.append(fc.ExtrusionGeometry(area_model='rectangle', width=EW, height=EH))

    init.append(fc.PrinterCommand(id='home'))
    init.append(fc.ManualGcode(text='G29 L0 ; load mesh from slot 0'))
    init.append(fc.ManualGcode(text='G29 F10 ; set fade height for correction at 10 mm'))
    init.append(fc.ManualGcode(text='G29 A ; activate UBL'))

    init.append(fc.Extruder(on=False))
    init.append(fc.Printer(travel_speed=travel_speed, print_speed=print_speed))  # TBD to confirm
    init.append(fc.Fan(speed_percent=0))
    init.append(fc.Hotend(temp=nozzle_temp, wait=False))
    init.append(fc.Buildplate(temp=bed_temp, wait=False))
    init.append(fc.Point(x=2, y=2, z=50))

    init.append(fc.Hotend(temp=nozzle_temp, wait=True))
    init.append(fc.Buildplate(temp=bed_temp, wait=True))
    init.append(fc.PrinterCommand(id='beep'))

    init.extend(gcode_banner('PRIMER LINES'))
    init.append(fc.Point(z=EH))
    init.append(fc.Extruder(on=True))
    init.append(fc.Point(y=150))
    init.append(fc.Point(x=2.4))
    init.append(fc.Point(y=2))
    init.append(fc.Extruder(on=False))
    #init.append(fc.PrinterCommand(id='retract'))  # TBD
    
    init.extend(gcode_banner('PRINTING STEPS'))
    
    ending = []
    ending.extend(gcode_banner('START OF ENDING PROCEDURE'))
    ending.append(fc.Extruder(on=False))
    ending.append(fc.ManualGcode(text="G91 ; Set all axes to relative"))
    ending.append(fc.ManualGcode(text=f"G0 Z50 ; Move up above design"))
    ending.append(fc.ManualGcode(text="G90 ; Set all axes to absolute"))
    ending.append(fc.Point(x=2, y=220))
    ending.append(fc.ManualGcode(text='G4 ; wait motion to complete'))
    ending.append(fc.ManualGcode(text='M107 ; Fan Off'))
    ending.append(fc.Hotend(temp=0, wait=False))
    ending.append(fc.Buildplate(temp=0, wait=False))
    ending.append(fc.ManualGcode(text='M84 X Y E ; disable motors (except Z)'))
    ending.append(fc.PrinterCommand(id='beep'))
    ending.extend(gcode_banner('END OF ENDING PROCEDURE'))
    
    gcode = fc.transform(
        init + steps + ending,
        'gcode',
        fc.GcodeControls(
            printer_name='custom',
            initialization_data={
                #'primer': 'travel',
                #'print_speed': print_speed,
                #'travel_speed': travel_speed,
                #'nozzle_temp': nozzle_temp,
                #'bed_temp': bed_temp,
                #'fan_percent': 0,
                'extrusion_width': EW,
                'extrusion_height': EH,
            },
            save_as=filename,
            include_date=False
        )
    )
    return gcode

### Output

In [ ]:
print(to_gcode(steps))